In [92]:
import numpy as np
import pandas as pd
import pyvisa
rm = pyvisa.ResourceManager()
rm.list_resources()

('USB0::0x1AB1::0x0960::DSA8A224501273::INSTR',)

In [93]:
from datetime import datetime

In [94]:
#Preseting before collect data
inst = rm.open_resource('USB0::0x1AB1::0x0960::DSA8A224501273::INSTR')
inst.write(":SENSe:FREQuency:CENTer 150000000") #set center frquency
#inst.write(":SENSe:FREQuency:SPAN 100000000") # can be set in here or set Start/Stop Frq
inst.write(":SENSe:BANDwidth:RESolution 1000000") # set RBW: 100kHz, 300kHz, 1MHz
inst.write(":SENSe:FREQuency:START 50000000") #Start Frq in HZ
inst.write(":SENSe:FREQuency:STOP 100000000") #Stop Frq in Hz
inst.write(":SENSe:SWEep:TIME 0.01") # set sweep times to 10 ms
print("Center Frq:",inst.query(":SENSe:FREQuency:CENTer?"))
print("Frq SPAN:",inst.query(":SENSe:FREQuency:SPAN?"))
print("Frq start:",inst.query(":SENSe:FREQuency:STARt?"))
print("Frq stop:",inst.query(":SENSe:FREQuency:STOP?"))
print("Sweep time:",inst.query(":SENSe:SWEep:TIME?"))
inst.close()

Center Frq: 75000000

Frq SPAN: 50000000

Frq start: 50000000

Frq stop: 100000000

Sweep time: 1.000000E-02



In [106]:
#colleting data
n=10                         # 10 times is 1 sec
for i in range (n):  
    rm = pyvisa.ResourceManager()
    inst = rm.open_resource('USB0::0x1AB1::0x0960::DSA8A224501273::INSTR')
    a=inst.query(":TRACe:DATA? TRACE1")
    data= a.split(",")
    b=data[0]                #convert first value to float for remove #9000009014 in first element
    data[0]=float(b[12:25])  #covert first value to float
    del data[600]            # remove NaN in last value then we get 600 point per round 
    df = pd.DataFrame(data)
    df.to_csv('D:/Thesis 4th year/spectrum analyzer/test'+str(i)+'ms.csv')
    #ISO_time = datetime.now().isoformat()
    #print(f"Current DateTime in ISO: {ISO_time}")
    #time.sleep(0.1)          # for each round it take around 100 ms per round
    
    inst.close()